In [31]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
import tensorflow.keras as keras
from keras.applications import ResNet50
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix , accuracy_score

In [32]:
#Model Parameters
batch_size = 8
learning_rate = 0.0001
EPOCHS =30

In [33]:
# Set the directory paths for the benign and malignant images
benign_dir = 'Dataset/Benign Masses'
malignant_dir = 'Dataset/Malignant Masses'

# Target size for resizing
target_size = (224, 224)

# Lists to store the images and labels
images = []
labels = []

# Function to resize and process the images
def process_image(image_path, label):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, target_size)
    images.append(resized_image)
    labels.append(label)

# Processing the benign images
for filename in os.listdir(benign_dir):
    image_path = os.path.join(benign_dir, filename)
    process_image(image_path, label='benign')

# Processing the malignant images
for filename in os.listdir(malignant_dir):
    image_path = os.path.join(malignant_dir, filename)
    process_image(image_path, label='malignant')

In [34]:
#Converting lists to numpy arrays
images = np.array(images)/255.0
labels = np.array(labels)

In [35]:
#Encoding labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
#One-hot-encoding our labels
labels = to_categorical(labels)

In [36]:
# Assuming X contains your feature data and y contains your target labels
trainX, testX, trainY, testY = train_test_split(images, labels, test_size=0.2, random_state=42)

In [37]:
#Data augmentation
trainAug = ImageDataGenerator(rotation_range=15, fill_mode = "nearest")

In [38]:
def build_model(backbone, learning_rate=1e-4):
    model = Sequential()
    model.add(backbone)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(2, activation='softmax'))
    
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )
    
    return model
    

In [39]:
resnet = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)


model = build_model(resnet ,learning_rate = 1e-4)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 2048)              0         
                                                                 
 batch_normalization_3 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23,600,002
Trainable params: 23,542,786


In [40]:
# Compile the model
opt = Adam  (learning_rate=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

H = model.fit(
    trainAug.flow(trainX, trainY, batch_size=batch_size),
    steps_per_epoch=trainX.shape[0] / batch_size,
    epochs=EPOCHS,
    validation_data = (testX, testY),
    )


# Evaluate the model
loss, accuracy = model.evaluate(testX, testY)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/30
29/29 [==============================] - 56s 2s/step - loss: 0.8004 - accuracy: 0.5517 - val_loss: 0.6963 - val_accuracy: 0.5172
Epoch 2/30
29/29 [==============================] - 43s 1s/step - loss: 0.5215 - accuracy: 0.7672 - val_loss: 0.7168 - val_accuracy: 0.5172
Epoch 3/30
29/29 [==============================] - 44s 2s/step - loss: 0.3489 - accuracy: 0.8922 - val_loss: 0.7917 - val_accuracy: 0.5172
Epoch 4/30
29/29 [==============================] - 46s 2s/step - loss: 0.3481 - accuracy: 0.8707 - val_loss: 0.6931 - val_accuracy: 0.5172
Epoch 5/30
29/29 [==============================] - 43s 1s/step - loss: 0.2713 - accuracy: 0.9009 - val_loss: 0.6981 - val_accuracy: 0.5345
Epoch 6/30
29/29 [==============================] - 43s 1s/step - loss: 0.2831 - accuracy: 0.8922 - val_loss: 0.7666 - val_accuracy: 0.4828
Epoch 7/30
29/29 [==============================] - 43s 1s/step - loss: 0.1355 - accuracy: 0.9569 - val_loss: 0.8394 - val_accuracy: 0.4828
Epoch 8/30
29/29 [==